Import necessary packages

In [23]:
from langchain_openai import OpenAI
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI
from pathlib import Path
from langchain_core.output_parsers import StrOutputParser
from langchain.output_parsers import PydanticToolsParser
import datetime
from typing import Literal, Optional, Tuple
from langchain_core.pydantic_v1 import BaseModel, Field


Function definitions

In [24]:
#function to call a path to the file and read it. 
def context_gen(file_name):
    Folder = "VertexLang"
    here = Path(locals().get('__file__', Folder)).resolve()
    parameter = (here / file_name).read_text()
    return parameter

#remove code fences from the output
def remove_code_fences(text):
    lines = text.split("\n")
    lines = [line for line in lines if not line.strip().startswith('```')]
    lines[0] = lines[0].replace(' -', '-', 1)
    return "\n".join(lines)

#invoke and run the model with the given prompt
def test_vertexai(few_shot_prompt):
    generate = ChatPromptTemplate.from_messages([("system", system), few_shot_prompt, ("human", human),])
    chat = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.5)
    #select an output parser
    output_parser = StrOutputParser()
    chain = generate | chat | output_parser
    result = chain.invoke({}, {"tags": ["loop 001"]})
    print(remove_code_fences(result))
    print()
    return


Model Inputs

In [25]:
#create inputs to the model, telling it what needs to be done. 
#provides system message
system = context_gen("systemmsg.txt")

#provides the few shot examples
output_examples = context_gen("outputex.txt")

#provides the input examples
input_examples = context_gen("inputex.txt")

#provides the database schema
schema = context_gen("dataBaseSchema.txt")

'''change the human variable to test different inputs'''
human = ("select the blend in the xz plane between the angle 45 and 90 and then select the connected blend")


In [26]:
class SubQuery(BaseModel):
    '''Search for the geometric definition of a feature across the internet'''

    sub_query: str = Field(
        ...,
        description="The text to be used as a sub-query in the prompt.",
    )   

In [31]:
import uuid 
from typing import Dict, List

from langchain_core.messages import (AIMessage, BaseMessage, HumanMessage, SystemMessage, ToolMessage)

def tool_example_to_message(example: Dict) -> List[BaseMessage]:
    messages: List[BaseMessage] = [HumanMessage(content=example["input"])]
    openai_tool_calls = []
    for tool_call in example["tool_calls"]:
        openai_tool_calls.append(
            {"id": str(uuid.uuid4()), "type": "function", 
             "function": {
                 "name": tool_call.__class__.__name__,
                 "arguments": tool_call.json(),                    
                        },
            }
        )
    messages.append(
    AIMessage(content="", additional_kwargs={"tool_calls": openai_tool_calls})
)
    tool_outputs = example.get("tool_outputs") or [
        "This is an example of a correct usage of this tool. Make sure to continue using the tool this way."
    ] * len(openai_tool_calls)
    for output, tool_call in zip(tool_outputs, openai_tool_calls):
        messages.append(ToolMessage(content=output, tool_call_id=tool_call["id"]))
    return messages

examples = []
user = "Select all boltholes"
queries = [
    SubQuery(sub_query="What is the definition of boltholes"),
]
examples.append({"input": user, "tool_calls": queries})

user = "select all vanes"
queries = [
    SubQuery(sub_query="What is the definition of vanes"),
]
examples.append({"input": user, "tool_calls": queries})

example_messages = [msg for ex in examples for msg in tool_example_to_message(ex)]

system2 = context_gen("system2.txt")
prompt = ChatPromptTemplate.from_messages([("system", system2), MessagesPlaceholder("examples", optional=True), ("human", human),])

chat = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.5)
chat_with_tools = chat.bind_tools([SubQuery])
parser = PydanticToolsParser(tools = [SubQuery])
query_analyzer = prompt | chat_with_tools | parser 
query_analyzer.invoke({}, {"tags": ["loop 001"]})

[SubQuery(sub_query='What is the geometric definition of a blend in the xz plane?'),
 SubQuery(sub_query='What is the geometric definition of connected blends in a geometric context?')]

Few shot prompts example template 

In [28]:
examples = [
    {"input": input_examples, "output": output_examples},
]

example_prompt = ChatPromptTemplate.from_messages([
    ("human", "{input}"),
    ("ai", "{output}"),
]
)

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

'''Function call to run the model'''
test_vertexai(few_shot_prompt)

- filter: select * from faces where type like "Blend" and (xyangle between 45 and 90)
- expand: connectedblend

